# De XML para CSV

O formato XML não facilita muito a vida de quem quer utilizar os dados de forma mais simples. Após descompactar os [Acórdãos do Segundo Grau](https://osf.io/as8uv/), o pesquisador fica com vários XMLs no diretório, totalizando cerca de 2.2G de dados. Com esse volume, já sentimos uma certa lentidão para realizar a carga dos arquivos, imagine fazer operações de agrupamento, ordenação, etc. Ou seja, temos muito material para 'brincar'!

Considerando que o formato CSV acabou tornando-se muito popular para disponibilizar dados na Web, então, vamos fazer a conversão! 

Em virtude do volume, poderíamos utilizar o [Dask](https://dask.org/), por exemplo, para paralelizar o parser dos arquivos, mas a ideia é começarmos simples e, em outros notebooks, explorarmos outros tópicos/tecnologias. 

O XML possui algumas colunas que não agregam valor para análise, haja vista que são dados utilizados internamente pelo motor de buscas do TJSE (o campo Chave, por exemplo). Sendo assim, utilizamos somente as colunas:
 * txtAcordao: Texto do acórdão;
 * data: Data de publicação do acórdão;
 * txtVoto: Texto relatando o voto de cada Desembargador;
 * NroProcesso: Número do processo; 
 * NroAcordao: Número do acórdão;
 * DescRecurso: Tipo de recurso;
 * DesRelator: Nome do relator do acórdão;
 * OrgaoJulgador: Órgão que julgou o acórdão;
 * txtEmenta: Texto da ementa. Este é um dos campos textuais mais importantes, pois representa um resumo do que foi tratado pelo acórdão; 
 * txtRelatorio: Texto do relatório do acórdão.
 
 
Para entender um pouco mais sobre o que é um acórdão recomendo a leitura [desse](https://www.infoescola.com/direito/acordao/) artigo, ou do Cap. 2 de [minha dissertação](http://osf.io/x3rbn/?action=download) se tiver com um pouco mais de tempo. :)
 

In [1]:
#Caso os XMLs não coubessem na memória, seria melhor ir gerando o CSV conforme o XML fosse lido.
#Como não é o caso aqui, carregaremos simplesmente os registros na memória.

from pathlib import Path
import xml.etree.ElementTree as ET
import numpy as np

acordaos_dict = {'txtAcordao':[], 
      'data':[], 
      'txtVoto':[], 
      'NroProcesso':[], 
      'NroAcordao':[], 
      'DescRecurso':[], 
      'DesRelator':[], 
      'OrgaoJulgador':[], 
      'txtEmenta':[], 
      'txtRelatorio':[]}

def popular_dicionario(arquivo, dicionario=None):    
    if dicionario is None:
        raise ValueError('É obrigatório usar um dicionário.')
        
    tree = ET.parse(arquivo)
    root = tree.getroot()
    for doc in tree.findall(".//doc"):
        for k in dicionario.keys():
            try:
                texto = doc.find(f"./field[@name='{k}']").text
                dicionario[k].append(texto)
            except:
                dicionario[k].append(np.nan)

xml_dir = Path("dataset/asg/")

for xml in xml_dir.glob('*.xml'):
    popular_dicionario(xml, acordaos_dict)

## Limpeza de Dados

Além de fazer a conversão, que tal aproveitarmos os dados em memória para executar alguns tratamentos nos dados? Para tal, utilizaremos o [Pandas](https://pandas.pydata.org/).

In [2]:
import pandas as pd

df = pd.DataFrame(acordaos_dict)
del acordaos_dict #Não precisamos mais do dicionário ocupando nossa preciosa memória
df.head()

,txtAcordao,data,txtVoto,NroProcesso,NroAcordao,DescRecurso,DesRelator,OrgaoJulgador,txtEmenta,txtRelatorio
0,"Vistos, relatados e discutidos estes autos, a...",2015-07-30T00:00:00Z,DESEMBARGADORA ELVIRA MARIA DE ALMEIDA SILVA ...,201500712507,201512285,Embargos de Declaração,Desa. Elvira Maria de Almeida Silva,1ª CÂMARA CÍVEL,EMBARGOS DECLARATÓRIOS ? INEXISTÊNCIA DE OMIS...,DESEMBARGADORA ELVIRA MARIA DE ALMEIDA SILVA ...
1,"Vistos, relatados e discutidos os presentes a...",2015-12-02T00:00:00Z,O recurso preencheu os requisitos de validade...,201500712508,201521198,Agravo de Instrumento,Des. Osório de Araújo Ramos Filho,1ª CÂMARA CÍVEL,PROCESSO CIVIL. AGRAVO DE INSTRUMENTO. EXECUÇ...,A EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇÃO ? ...
2,"Vistos, relatados e discutidos estes autos, a...",2015-06-25T00:00:00Z,Desa. Elvira Maria de Almeida Silva (Relatora...,201500712512,20159547,Apelação,Desa. Elvira Maria de Almeida Silva,1ª CÂMARA CÍVEL,APELAÇÃO CÍVEL. AÇÃO DE USUCAPIÃO.EXTINÇÃO DO...,R E L A T Ó R I O Desembargadora Elvira Maria...
3,"Vistos, relatados e discutidos os presentes a...",2015-11-03T00:00:00Z,"Preenchidos os requisitos de admissibilidade,...",201500712515,201518637,Apelação,Des. Roberto Eugenio da Fonseca Porto,1ª CÂMARA CÍVEL,APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANO...,Cuida-se de Apelações Cíveis interposta pela ...
4,"Vistos, relatados e discutidos os presentes a...",2015-06-19T00:00:00Z,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...,201500712518,20159286,Apelação,Des. Roberto Eugenio da Fonseca Porto,1ª CÂMARA CÍVEL,APELAÇÃO CÍVEL ? JULGAMENTO DE IMPUGNAÇÃO COM...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181994 entries, 0 to 181993
Data columns (total 10 columns):
txtAcordao       181935 non-null object
data             181994 non-null object
txtVoto          181994 non-null object
NroProcesso      181994 non-null object
NroAcordao       181994 non-null object
DescRecurso      181993 non-null object
DesRelator       181720 non-null object
OrgaoJulgador    181994 non-null object
txtEmenta        181994 non-null object
txtRelatorio     172528 non-null object
dtypes: object(10)
memory usage: 13.9+ MB


Beleza! Temos um dataframe com 181994 acórdãos. Vamos começar dando uma olhada nos nomes dos relatores. Será que tem nomes iguais com *cases* diferentes?

In [4]:
display(df.DesRelator.unique().size - df.DesRelator.str.lower().unique().size)

11

Acertamos no palpite! Vamos converter todos os nomes para *lower case* e ver o que acontece.

In [5]:
df.DesRelator = df.DesRelator.str.lower()

df.DesRelator.sort_values().unique()

array(['corregedor geral de justiça', 'des. alberto romeu gouveia leite',
       'des. aloiso de abreu lima', 'des. artur oscar de oliveira deda',
       'des. cezário siqueira neto', 'des. cláudio dinart déda chagas',
       'des. diógenes barreto', 'des. edson ulisses de melo',
       'des. epaminondas s. de andrade lima',
       'des. fernando ribeiro franco', 'des. gilson gois soares',
       'des. jose antonio de a. goes', 'des. jose barreto prado',
       'des. jose nolasco de carvalho', 'des. josé alves neto',
       'des. josé artêmio barreto', 'des. josé dos anjos',
       'des. luiz antônio araújo mendonça', 'des. manoel cândido filho',
       'des. manuel pascoal nabuco d`avila',
       'des. netônio bezerra machado',
       'des. osório de araújo ramos filho',
       'des. ricardo múcio santana de a. lima',
       'des. ricardo múcio santana de abreu lima',
       'des. roberto eugenio da fonseca porto',
       'des. ruy pinheiro da silva',
       'desa. ana lúcia freire de

Os nomes 'des. ricardo múcio santana de a. lima' e 'des. ricardo múcio santana de abreu lima' representam a mesma pessoa. Para economizar alguns *keystrokes*, manteremos somente a versão abreviada:

In [6]:
df.DesRelator = df.DesRelator.replace('des. ricardo múcio santana de abreu lima', 'des. ricardo múcio santana de a. lima')

Aparentemente, os nomes estão ok. Para evitar situações desse tipo, vamos converter `DescRecurso` e `OrgaoJulgador` para caixa-baixa.

In [7]:
df.DescRecurso = df.DescRecurso.str.lower()
display(len(df.DescRecurso.unique()))
display(df.DescRecurso.drop_duplicates().head()) #visualizando somente os primeiros tipos de recurso

df.OrgaoJulgador = df.OrgaoJulgador.str.lower()
df.OrgaoJulgador.unique() #aqui são poucos, podemos visualizar todos


141

0             embargos de declaração
1              agravo de instrumento
2                           apelação
45                 agravo regimental
380    apelação / remessa necessária
Name: DescRecurso, dtype: object

array(['1ª câmara cível', '2ª câmara cível', 'câmara criminal',
       'câmara cível', 'conselho da magistratura',
       'câmaras cíveis reunidas', 'tribunal pleno'], dtype=object)

Massa! Vamos olhar agora o campo `data`. Já que nenhuma data está com o valor nulo, vamos checar se todas estão no formato correto.

In [8]:
idx_formato_correto = df.data.str.match(r'\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}Z')
len(df) - sum(idx_formato_correto)

0

Ótimo! Todas as datas estão com formato válido. Vamos ver se o Pandas consegue convertê-las:

In [9]:
df.data = pd.to_datetime(df.data)

O CSV não armazena informações de tipo (o [Parquet](https://parquet.apache.org/) está ganhando popularidade e esse é um dos motivos), mas, pelo menos, já sabemos que o Pandas está conseguindo converter sem problemas. Vamos dar mais uma olhada no nosso dataframe:

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181994 entries, 0 to 181993
Data columns (total 10 columns):
txtAcordao       181935 non-null object
data             181994 non-null datetime64[ns, UTC]
txtVoto          181994 non-null object
NroProcesso      181994 non-null object
NroAcordao       181994 non-null object
DescRecurso      181993 non-null object
DesRelator       181720 non-null object
OrgaoJulgador    181994 non-null object
txtEmenta        181994 non-null object
txtRelatorio     172528 non-null object
dtypes: datetime64[ns, UTC](1), object(9)
memory usage: 13.9+ MB


,txtAcordao,data,txtVoto,NroProcesso,NroAcordao,DescRecurso,DesRelator,OrgaoJulgador,txtEmenta,txtRelatorio
0,"Vistos, relatados e discutidos estes autos, a...",2015-07-30 00:00:00+00:00,DESEMBARGADORA ELVIRA MARIA DE ALMEIDA SILVA ...,201500712507,201512285,embargos de declaração,desa. elvira maria de almeida silva,1ª câmara cível,EMBARGOS DECLARATÓRIOS ? INEXISTÊNCIA DE OMIS...,DESEMBARGADORA ELVIRA MARIA DE ALMEIDA SILVA ...
1,"Vistos, relatados e discutidos os presentes a...",2015-12-02 00:00:00+00:00,O recurso preencheu os requisitos de validade...,201500712508,201521198,agravo de instrumento,des. osório de araújo ramos filho,1ª câmara cível,PROCESSO CIVIL. AGRAVO DE INSTRUMENTO. EXECUÇ...,A EMPRESA MUNICIPAL DE OBRAS E URBANIZAÇÃO ? ...
2,"Vistos, relatados e discutidos estes autos, a...",2015-06-25 00:00:00+00:00,Desa. Elvira Maria de Almeida Silva (Relatora...,201500712512,20159547,apelação,desa. elvira maria de almeida silva,1ª câmara cível,APELAÇÃO CÍVEL. AÇÃO DE USUCAPIÃO.EXTINÇÃO DO...,R E L A T Ó R I O Desembargadora Elvira Maria...
3,"Vistos, relatados e discutidos os presentes a...",2015-11-03 00:00:00+00:00,"Preenchidos os requisitos de admissibilidade,...",201500712515,201518637,apelação,des. roberto eugenio da fonseca porto,1ª câmara cível,APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANO...,Cuida-se de Apelações Cíveis interposta pela ...
4,"Vistos, relatados e discutidos os presentes a...",2015-06-19 00:00:00+00:00,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...,201500712518,20159286,apelação,des. roberto eugenio da fonseca porto,1ª câmara cível,APELAÇÃO CÍVEL ? JULGAMENTO DE IMPUGNAÇÃO COM...,Des. ROBERTO EUGENIO DA FONSECA PORTO (Relato...


Poderíamos também converter os campos `txtAcordao`, `txtEmenta` e `txtRelatório` para caixa baixa e, além disso, remover pontuação, quebras de linhas, etc. No entanto, não queremos eliminar nenhuma possibilidade de análise de forma precipitada. De repente, o pesquisador descobre que caixa-alta dentro dos textos representa algum tipo de cabeçalho (será? não sei) e quer fazer uma análise em cima disso. Portanto, deixemos assim mesmo.

Agora podemos gerar nosso arquivo CSV:

In [11]:
df.to_csv('dataset/asg.csv.gz', index=False)

O arquivo ficou com 638M. Ah! O read_csv do pandas consegue ler arquivos comprimidos, não precisa decompactá-los. Agora podemos aposentar esses arquivos XML. Nos próximos notebooks usaremos somente esse arquivo CSV gerado.

Disponibilizei o arquivo para [download](https://osf.io/4daqk/?action=download).